In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [29]:
# Importar las librerías necesarias
import pandas as pd
import numpy as np

# Preprocesamiento y modelado
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier

# Manejo del desbalance de clases
from imblearn.over_sampling import SMOTE

# Métricas de evaluación
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

# Ignorar advertencias
import warnings
warnings.filterwarnings('ignore')

# Paso 1: Cargar datos desde un archivo CSV
df = pd.read_csv('/content/drive/MyDrive/dataset.csv')

# Paso 2: División de datos de entrenamiento y prueba
X = df['name']
y = df['category']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Paso 3: Eliminar filas con NaN en y_train y sus correspondientes en X_train
train_data = pd.DataFrame({'name': X_train, 'category': y_train})
train_data = train_data.dropna()
X_train = train_data['name']
y_train = train_data['category']

# Paso 4: Vectorización usando TF-IDF (análisis de caracteres n-grama)
vectorizer = TfidfVectorizer(analyzer='char', ngram_range=(2, 4))
X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

# Paso 5: Ingeniería de características adicionales
# Funciones auxiliares para calcular características
def count_digits(s):
    return sum(c.isdigit() for c in s)

def count_special_chars(s):
    return sum(not c.isalnum() for c in s)

def vowel_consonant_ratio(s):
    vowels = 'aeiouAEIOU'
    num_vowels = sum(c in vowels for c in s)
    num_consonants = sum(c.isalpha() and c not in vowels for c in s)
    return num_vowels / num_consonants if num_consonants != 0 else 0

# Calcular características adicionales para el conjunto de entrenamiento
X_train_length = X_train.apply(len)
X_train_digit_count = X_train.apply(count_digits)
X_train_special_count = X_train.apply(count_special_chars)
X_train_vowel_ratio = X_train.apply(vowel_consonant_ratio)

# Calcular características adicionales para el conjunto de prueba
X_test_length = X_test.apply(len)
X_test_digit_count = X_test.apply(count_digits)
X_test_special_count = X_test.apply(count_special_chars)
X_test_vowel_ratio = X_test.apply(vowel_consonant_ratio)

# Combinar las características adicionales en matrices numpy
additional_features_train = np.vstack([X_train_length, X_train_digit_count, X_train_special_count, X_train_vowel_ratio]).T
additional_features_test = np.vstack([X_test_length, X_test_digit_count, X_test_special_count, X_test_vowel_ratio]).T

# Combinar las características TF-IDF con las adicionales
from scipy.sparse import hstack, csr_matrix

X_train_combined = hstack([X_train_tfidf, csr_matrix(additional_features_train)])
X_test_combined = hstack([X_test_tfidf, csr_matrix(additional_features_test)])

# Paso 6: Manejo del desbalance de clases usando SMOTE
smote = SMOTE(random_state=42)
X_train_resampled, y_train_resampled = smote.fit_resample(X_train_combined, y_train)

# Paso 7: Entrenamiento del modelo con Random Forest ajustando class_weight
model = RandomForestClassifier(n_estimators=200, class_weight='balanced', random_state=42)
model.fit(X_train_resampled, y_train_resampled)

# Paso 8: Predicciones y evaluación del modelo
y_pred = model.predict(X_test_combined)

# Métricas de evaluación
print("Accuracy:", accuracy_score(y_test, y_pred))
print("\nClassification Report:\n", classification_report(y_test, y_pred))
print("Matriz de Confusión:\n", confusion_matrix(y_test, y_pred))

# Paso 9: Validación cruzada
scores = cross_val_score(model, X_train_resampled, y_train_resampled, cv=5, scoring='f1_macro')
print("\nF1-score promedio en validación cruzada:", scores.mean())

# Paso 10: Análisis de importancia de características
importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

print("\nLas 10 características más importantes:")
feature_names = vectorizer.get_feature_names_out()
num_tfidf_features = len(feature_names)

for f in range(10):
    if indices[f] < num_tfidf_features:
        feature_name = feature_names[indices[f]]
    else:
        additional_index = indices[f] - num_tfidf_features
        if additional_index == 0:
            feature_name = 'length'
        elif additional_index == 1:
            feature_name = 'digit_count'
        elif additional_index == 2:
            feature_name = 'special_char_count'
        elif additional_index == 3:
            feature_name = 'vowel_consonant_ratio'
        else:
            feature_name = 'unknown'
    print(f"{f + 1}. Feature '{feature_name}' ({importances[indices[f]]})")

# Paso 11: Función para probar el modelo con nuevos nombres
def classify_names(names):
    # Asegurarse de que 'names' sea una lista
    if isinstance(names, str):
        names = [names]
    elif not isinstance(names, list):
        names = list(names)

    # Vectorización de los nuevos nombres
    names_tfidf = vectorizer.transform(names)

    # Cálculo de características adicionales
    names_length = [len(name) for name in names]
    names_digit_count = [count_digits(name) for name in names]
    names_special_count = [count_special_chars(name) for name in names]
    names_vowel_ratio = [vowel_consonant_ratio(name) for name in names]

    additional_features = np.vstack([names_length, names_digit_count, names_special_count, names_vowel_ratio]).T

    # Combinar características
    names_combined = hstack([names_tfidf, csr_matrix(additional_features)])

    # Realizar predicciones
    predictions = model.predict(names_combined)

    # Devolver resultados
    for name, prediction in zip(names, predictions):
        print(f"Nombre: '{name}' => Clasificación: '{prediction}'")

# Ejemplos de uso de la función classify_names
print("\nPruebas con nuevos nombres:")
test_names = [
    'skrillex',
    'dourat',
    'dragonxx',
    'ywhuhsus8nj',
    'opojwrjwjos12',
    'blackmamba',
    'xj92kdkls',
    'phantomfox',
    'abc123',
    'xypqrs',
    'supermalware',
    'randomname'
]

classify_names(test_names)



Accuracy: 0.9250585480093677

Classification Report:
               precision    recall  f1-score   support

     generic       0.95      0.93      0.94       254
         new       0.89      0.92      0.91       173

    accuracy                           0.93       427
   macro avg       0.92      0.93      0.92       427
weighted avg       0.93      0.93      0.93       427

Matriz de Confusión:
 [[235  19]
 [ 13 160]]

F1-score promedio en validación cruzada: 0.9563726650905217

Las 10 características más importantes:
1. Feature 'digit_count' (0.05357759759344331)
2. Feature 'vowel_consonant_ratio' (0.0456245067492046)
3. Feature 'length' (0.04267565790545367)
4. Feature 'er' (0.01474898477198885)
5. Feature 'ra' (0.010473824151821244)
6. Feature 'in' (0.00964771497855725)
7. Feature 'at' (0.008926079416604815)
8. Feature 'mr' (0.008743719478806106)
9. Feature 'ff' (0.008360634916843438)
10. Feature 'or' (0.008312116291122938)

Pruebas con nuevos nombres:
Nombre: 'skrillex' => Clas

In [33]:

# Paso 11: Función para probar el modelo con nuevos nombres
def classify_names(names):
    # Asegurarse de que 'names' sea una lista
    if isinstance(names, str):
        names = [names]
    elif not isinstance(names, list):
        names = list(names)

    # Vectorización de los nuevos nombres
    names_tfidf = vectorizer.transform(names)

    # Cálculo de características adicionales
    names_length = [len(name) for name in names]
    names_digit_count = [count_digits(name) for name in names]
    names_special_count = [count_special_chars(name) for name in names]
    names_vowel_ratio = [vowel_consonant_ratio(name) for name in names]

    additional_features = np.vstack([names_length, names_digit_count, names_special_count, names_vowel_ratio]).T

    # Combinar características
    names_combined = hstack([names_tfidf, csr_matrix(additional_features)])

    # Realizar predicciones
    predictions = model.predict(names_combined)

    # Devolver resultados
    for name, prediction in zip(names, predictions):
        print(f"Nombre: '{name}' => Clasificación: '{prediction}'")

# Ejemplos de uso de la función classify_names
print("\nPruebas con nuevos nombres:")
test_names = [
    'uwywquyuwqs125',
    'federat',
    'ltq0',
    'souldarker',
    'noolsddos',
    'rtto',
    'nvsrimm',
    'tu1qwtuxxwooknxbsinzhupkxxjn',
    'fiummmm'
]

classify_names(test_names)



Pruebas con nuevos nombres:
Nombre: 'uwywquyuwqs125' => Clasificación: 'generic'
Nombre: 'federat' => Clasificación: 'new'
Nombre: 'ltq0' => Clasificación: 'generic'
Nombre: 'souldarker' => Clasificación: 'new'
Nombre: 'noolsddos' => Clasificación: 'new'
Nombre: 'rtto' => Clasificación: 'generic'
Nombre: 'nvsrimm' => Clasificación: 'generic'
Nombre: 'tu1qwtuxxwooknxbsinzhupkxxjn' => Clasificación: 'generic'
Nombre: 'fiummmm' => Clasificación: 'new'
